# Mining ASTER Analysis

## Objectives
- **Alteration mapping** (Clay, Iron, Carbonates)
- **Alteration-based mineralization potential** (spectral signatures only)
- Temporal comparison (2001-2006 vs 2023-2024)
- Integration with SRTM 30 DEM

## Note on Terminology
This notebook focuses on **alteration mapping** and **alteration-based potential**. 
For final **mineralization maps** that incorporate structural controls (lineaments), 
see `mining-integrated.ipynb` which combines:
- Alteration (from this notebook)
- Structural features/lineaments (from `mining-palsar.ipynb`)

## Study Area
- Coordinates (WGS1984):
  - Point 1: 04°46'0.00"N, 29°34'30.00"E
  - Point 2: 04°46'0.00"N, 29°16'45.00"E
  - Point 3: 05°05'30.00"N, 29°16'45.00"E
  - Point 4: 05°05'30.00"N, 29°34'30.00"E
- Buffer: 20m


In [8]:
pip install geemap earthengine-api numpy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import required libraries
import geemap
import ee
import numpy as np
from datetime import datetime

# Initialize Earth Engine with Project ID
# IMPORTANT: Replace 'your-project-id' with your actual Google Cloud Project ID
# To find your Project ID:
# 1. Go to https://console.cloud.google.com/
# 2. Click on the project dropdown at the top
# 3. Copy the Project ID (e.g., 'ee-your-username' or 'mineral-exploration-456')

PROJECT_ID = 'ee-okwaretom12'  # <-- REPLACE THIS WITH YOUR PROJECT ID

try:
    # Try to initialize with project ID
    geemap.ee_initialize(project=PROJECT_ID)
    print(f"Earth Engine initialized with project: {PROJECT_ID}")
except Exception as e:
    print("Earth Engine not initialized. Starting authentication...")
    print("Please follow the authentication steps:")
    print("1. A browser window will open")
    print("2. Sign in with your Google account")
    print("3. Grant permissions to Earth Engine")
    print("4. Copy the authorization code and paste it when prompted")
    
    # Authenticate (this will open a browser for first-time users)
    ee.Authenticate()
    
    # Initialize after authentication with project ID
    geemap.ee_initialize(project=PROJECT_ID)
    print(f"Earth Engine initialized successfully with project: {PROJECT_ID}!")

# Initialize geemap
Map = geemap.Map()
print("Libraries imported and geemap initialized successfully!")


Earth Engine initialized with project: your-project-id
Libraries imported and geemap initialized successfully!


## Define Study Area


In [10]:
# Define study area coordinates (WGS1984)
# Convert DMS to decimal degrees
coords = [
    [29.575, 4.766667],  # Point 1: 29°34'30"E, 04°46'0"N
    [29.279167, 4.766667],  # Point 2: 29°16'45"E, 04°46'0"N
    [29.279167, 5.091667],  # Point 3: 29°16'45"E, 05°05'30"N
    [29.575, 5.091667],  # Point 4: 29°34'30"E, 05°05'30"N
]

# Create polygon
study_area = ee.Geometry.Polygon([coords])

# Apply 20m buffer
study_area_buffered = study_area.buffer(20)

# Print study area info (without .getInfo() to avoid blocking)
print("Study area created with coordinates:")
print("  Point 1: 29°34'30\"E, 04°46'0\"N")
print("  Point 2: 29°16'45\"E, 04°46'0\"N")
print("  Point 3: 29°16'45\"E, 05°05'30\"N")
print("  Point 4: 29°34'30\"E, 05°05'30\"N")
print("  Buffer: 20m")

# Add to map
Map.addLayer(study_area_buffered, {'color': 'red'}, 'Study Area (20m buffer)')
Map.centerObject(study_area_buffered, 10)
Map


Study area created with coordinates:
  Point 1: 29°34'30"E, 04°46'0"N
  Point 2: 29°16'45"E, 04°46'0"N
  Point 3: 29°16'45"E, 05°05'30"N
  Point 4: 29°34'30"E, 05°05'30"N
  Buffer: 20m


Map(center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position', 'transparent_b…

## Load SRTM 30 DEM


In [11]:
# Load SRTM 30m DEM
srtm = ee.Image("USGS/SRTMGL1_003").clip(study_area_buffered)

# Calculate slope and aspect for lineament analysis
elevation = srtm.select('elevation')
slope = ee.Terrain.slope(elevation)
aspect = ee.Terrain.aspect(elevation)
hillshade = ee.Terrain.hillshade(elevation)

# Add to map
Map.addLayer(elevation, {'min': 0, 'max': 2000, 'palette': ['blue', 'green', 'yellow', 'red']}, 'SRTM Elevation')
Map.addLayer(slope, {'min': 0, 'max': 45, 'palette': ['white', 'brown']}, 'Slope')
Map.addLayer(hillshade, {'min': 0, 'max': 255}, 'Hillshade', False)

print("SRTM DEM loaded successfully!")
Map


SRTM DEM loaded successfully!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

## Load ASTER Data - Period 1 (2001-2006)


In [12]:
# Load ASTER L1T data for period 2001-2006
aster_collection_2001_2006 = ee.ImageCollection('ASTER/AST_L1T_003') \
    .filterBounds(study_area_buffered) \
    .filterDate('2001-01-01', '2006-12-31') \
    .filter(ee.Filter.lt('CLOUDCOVER', 20))  # Filter clouds

print(f"Number of ASTER images (2001-2006): {aster_collection_2001_2006.size().getInfo()}")

# Create median composite to reduce noise
aster_2001_2006 = aster_collection_2001_2006.median().clip(study_area_buffered)

# ASTER bands: B01-B09 (VNIR and SWIR)
# B01: 0.52-0.60 μm (Green)
# B02: 0.63-0.69 μm (Red)
# B3N: 0.76-0.86 μm (NIR)
# B04: 1.60-1.70 μm (SWIR)
# B05: 2.145-2.185 μm (SWIR)
# B06: 2.185-2.225 μm (SWIR)
# B07: 2.235-2.285 μm (SWIR)
# B08: 2.295-2.365 μm (SWIR)
# B09: 2.360-2.430 μm (SWIR)

# Add RGB composite
Map.addLayer(aster_2001_2006, {
    'bands': ['B3N', 'B02', 'B01'],
    'min': 0,
    'max': 255
}, 'ASTER RGB (2001-2006)')

Map


Number of ASTER images (2001-2006): 13


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

## Load ASTER Data - Period 2 (2023-2024)


In [13]:
# Load ASTER L1T data for period 2023-2024
aster_collection_2023_2024 = ee.ImageCollection('ASTER/AST_L1T_003') \
    .filterBounds(study_area_buffered) \
    .filterDate('2023-01-01', '2024-12-31') \
    .filter(ee.Filter.lt('CLOUDCOVER', 20))

print(f"Number of ASTER images (2023-2024): {aster_collection_2023_2024.size().getInfo()}")

# Create median composite
aster_2023_2024 = aster_collection_2023_2024.median().clip(study_area_buffered)

# Add RGB composite
Map.addLayer(aster_2023_2024, {
    'bands': ['B3N', 'B02', 'B01'],
    'min': 0,
    'max': 255
}, 'ASTER RGB (2023-2024)')

Map


Number of ASTER images (2023-2024): 24


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

## Alteration Mapping - Clay Minerals


In [14]:
# Clay Mineral Index (CMI) using ASTER bands
# Clay minerals have strong absorption in SWIR bands
# CMI = (B06 + B07) / (B05 + B08)

def calculate_clay_index(image):
    """Calculate Clay Mineral Index"""
    b05 = image.select('B05').float()
    b06 = image.select('B06').float()
    b07 = image.select('B07').float()
    b08 = image.select('B08').float()
    
    # Clay Mineral Index
    clay_index = b06.add(b07).divide(b05.add(b08)).rename('Clay_Index')
    
    # AlOH Group Index (for clay detection)
    aloh_index = b05.divide(b06).rename('AlOH_Index')
    
    # Kaolinite Index
    kaolinite_index = b07.divide(b05).rename('Kaolinite_Index')
    
    return image.addBands([clay_index, aloh_index, kaolinite_index])

# Apply to both time periods
aster_2001_2006_clay = calculate_clay_index(aster_2001_2006)
aster_2023_2024_clay = calculate_clay_index(aster_2023_2024)

# Visualize clay indices
Map.addLayer(aster_2001_2006_clay.select('Clay_Index'), {
    'min': 0.8,
    'max': 1.2,
    'palette': ['blue', 'cyan', 'yellow', 'red']
}, 'Clay Index (2001-2006)')

Map.addLayer(aster_2023_2024_clay.select('Clay_Index'), {
    'min': 0.8,
    'max': 1.2,
    'palette': ['blue', 'cyan', 'yellow', 'red']
}, 'Clay Index (2023-2024)')

print("Clay mineral indices calculated!")
Map


Clay mineral indices calculated!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

## Alteration Mapping - Iron Oxides


In [15]:
# Iron Oxide Index using ASTER bands
# Iron oxides have strong absorption in visible bands
# Iron Index = B02 / B01

def calculate_iron_index(image):
    """Calculate Iron Oxide Index"""
    b01 = image.select('B01').float()
    b02 = image.select('B02').float()
    b3n = image.select('B3N').float()
    b04 = image.select('B04').float()
    b05 = image.select('B05').float()
    
    # Iron Oxide Index
    iron_index = b02.divide(b01).rename('Iron_Index')
    
    # Ferrous Iron Index
    ferrous_index = b05.divide(b04).rename('Ferrous_Index')
    
    # Ferric Iron Index
    ferric_index = b3n.divide(b01).rename('Ferric_Index')
    
    return image.addBands([iron_index, ferrous_index, ferric_index])

# Apply to both time periods
aster_2001_2006_iron = calculate_iron_index(aster_2001_2006)
aster_2023_2024_iron = calculate_iron_index(aster_2023_2024)

# Visualize iron indices
Map.addLayer(aster_2001_2006_iron.select('Iron_Index'), {
    'min': 0.8,
    'max': 1.5,
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']
}, 'Iron Index (2001-2006)')

Map.addLayer(aster_2023_2024_iron.select('Iron_Index'), {
    'min': 0.8,
    'max': 1.5,
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']
}, 'Iron Index (2023-2024)')

print("Iron oxide indices calculated!")
Map


Iron oxide indices calculated!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

In [16]:
# Carbonate Index using ASTER bands
# Carbonates have characteristic absorption in SWIR bands
# Carbonate Index = B06 / B07

def calculate_carbonate_index(image):
    """Calculate Carbonate Index"""
    b05 = image.select('B05').float()
    b06 = image.select('B06').float()
    b07 = image.select('B07').float()
    b08 = image.select('B08').float()
    
    # Carbonate Index
    carbonate_index = b06.divide(b07).rename('Carbonate_Index')
    
    # Calcite Index
    calcite_index = b08.divide(b07).rename('Calcite_Index')
    
    # Dolomite Index
    dolomite_index = b05.divide(b06).rename('Dolomite_Index')
    
    return image.addBands([carbonate_index, calcite_index, dolomite_index])

# Apply to both time periods
aster_2001_2006_carbonate = calculate_carbonate_index(aster_2001_2006)
aster_2023_2024_carbonate = calculate_carbonate_index(aster_2023_2024)

# Visualize carbonate indices
Map.addLayer(aster_2001_2006_carbonate.select('Carbonate_Index'), {
    'min': 0.9,
    'max': 1.1,
    'palette': ['blue', 'cyan', 'white', 'pink', 'red']
}, 'Carbonate Index (2001-2006)')

Map.addLayer(aster_2023_2024_carbonate.select('Carbonate_Index'), {
    'min': 0.9,
    'max': 1.1,
    'palette': ['blue', 'cyan', 'white', 'pink', 'red']
}, 'Carbonate Index (2023-2024)')

print("Carbonate indices calculated!")
Map


Carbonate indices calculated!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

## Composite Alteration Map


In [17]:
# Create composite alteration maps combining all three mineral types
def create_alteration_composite(clay_img, iron_img, carbonate_img):
    """Combine clay, iron, and carbonate indices into RGB composite"""
    # Normalize indices to 0-255 range for visualization
    clay_norm = clay_img.select('Clay_Index').multiply(255).byte()
    iron_norm = iron_img.select('Iron_Index').multiply(255).byte()
    carbonate_norm = carbonate_img.select('Carbonate_Index').multiply(255).byte()
    
    # Create RGB composite: R=Iron, G=Clay, B=Carbonate
    composite = ee.Image.cat([iron_norm, clay_norm, carbonate_norm]).rename(['R', 'G', 'B'])
    
    return composite

# Create composites for both periods
alteration_2001_2006 = create_alteration_composite(
    aster_2001_2006_clay,
    aster_2001_2006_iron,
    aster_2001_2006_carbonate
)

alteration_2023_2024 = create_alteration_composite(
    aster_2023_2024_clay,
    aster_2023_2024_iron,
    aster_2023_2024_carbonate
)

# Visualize composite alteration maps
Map.addLayer(alteration_2001_2006, {
    'min': 0,
    'max': 255
}, 'Alteration Composite (2001-2006) - R:Iron G:Clay B:Carbonate')

Map.addLayer(alteration_2023_2024, {
    'min': 0,
    'max': 255
}, 'Alteration Composite (2023-2024) - R:Iron G:Clay B:Carbonate')

print("Composite alteration maps created!")
Map


Composite alteration maps created!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

## Mineralisation Analysis


In [ ]:
# Create Alteration-Based Mineralization Potential Map
# This combines alteration indices to identify potential mineralization zones
# NOTE: This is based on spectral signatures only (alteration mapping)
# For final mineralization maps incorporating structural controls, see mining-integrated.ipynb

def create_alteration_potential_map(clay_img, iron_img, carbonate_img, study_area):
    """
    Create alteration-based mineralization potential map
    
    This map represents mineralization potential based on spectral alteration signatures only.
    It does NOT incorporate structural controls (lineaments, faults, fractures).
    
    For final mineralization maps that include structural controls, use mining-integrated.ipynb
    which combines alteration + structural features following standard practice in literature.
    """
    # Normalize and combine indices
    clay_prob = clay_img.select('Clay_Index').subtract(0.8).multiply(10).clamp(0, 1)
    iron_prob = iron_img.select('Iron_Index').subtract(0.8).multiply(5).clamp(0, 1)
    carbonate_prob = carbonate_img.select('Carbonate_Index').subtract(0.9).multiply(10).clamp(0, 1)
    
    # Weighted combination (adjust weights as needed)
    alteration_potential = clay_prob.multiply(0.4).add(
        iron_prob.multiply(0.3)
    ).add(
        carbonate_prob.multiply(0.3)
    ).rename('Alteration_Potential')
    
    # Classify into zones using percentile-based thresholds
    # Calculate percentiles for the study area
    stats = alteration_potential.reduceRegion(
        reducer=ee.Reducer.percentile([70, 80, 90, 95]),
        geometry=study_area,
        scale=30,
        maxPixels=1e9
    )
    
    # Get percentile thresholds (server-side)
    p70 = ee.Number(stats.get('p70'))
    p80 = ee.Number(stats.get('p80'))
    p90 = ee.Number(stats.get('p90'))
    p95 = ee.Number(stats.get('p95'))
    
    # Classify into confidence levels
    very_high = alteration_potential.gte(p95).rename('Very_High_Confidence_Zones')
    high = alteration_potential.gte(p90).And(alteration_potential.lt(p95)).rename('High_Confidence_Zones')
    medium = alteration_potential.gte(p80).And(alteration_potential.lt(p90)).rename('Medium_Confidence_Zones')
    low = alteration_potential.gte(p70).And(alteration_potential.lt(p80)).rename('Low_Confidence_Zones')
    all_zones = alteration_potential.gte(p70).rename('All_Alteration_Zones')
    alteration_zones = alteration_potential.gte(p90).rename('Alteration_Zones')  # Legacy support
    
    return alteration_potential.addBands([
        very_high, high, medium, low, all_zones, alteration_zones
    ])

# Create alteration-based potential maps for both periods
alteration_potential_2001_2006 = create_alteration_potential_map(
    aster_2001_2006_clay,
    aster_2001_2006_iron,
    aster_2001_2006_carbonate,
    study_area_buffered
)

alteration_potential_2023_2024 = create_alteration_potential_map(
    aster_2023_2024_clay,
    aster_2023_2024_iron,
    aster_2023_2024_carbonate,
    study_area_buffered
)

# Keep legacy variable names for backward compatibility with integrated notebook
mineralisation_2023_2024 = alteration_potential_2023_2024.select('Alteration_Potential').rename('Mineralisation_Probability')
mineralisation_2001_2006 = alteration_potential_2001_2006.select('Alteration_Potential').rename('Mineralisation_Probability')

# Visualize Alteration-Based Potential with improved confidence levels
print("=" * 60)
print("ALTERATION-BASED MINERALIZATION POTENTIAL")
print("=" * 60)
print("Using percentile-based thresholds (Top 5%, 10%, 20%, 30%)")
print("This represents potential based on SPECTRAL ALTERATION ONLY")
print("For final mineralization maps with structural controls, see mining-integrated.ipynb")
print("=" * 60)

# Continuous potential map
Map.addLayer(alteration_potential_2023_2024.select('Alteration_Potential'), {
    'min': 0,
    'max': 1,
    'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
}, 'Alteration-Based Potential (2023-2024)')

# Very High Confidence (Top 5% - Most Significant Targets)
Map.addLayer(alteration_potential_2023_2024.select('Very_High_Confidence_Zones'), {
    'min': 0,
    'max': 1,
    'palette': ['transparent', 'darkred']
}, 'Very High Confidence (Top 5%)', False)

# High Confidence (Top 10%)
Map.addLayer(alteration_potential_2023_2024.select('High_Confidence_Zones'), {
    'min': 0,
    'max': 1,
    'palette': ['transparent', 'red']
}, 'High Confidence (Top 10%)', False)

# Medium Confidence (Top 20%)
Map.addLayer(alteration_potential_2023_2024.select('Medium_Confidence_Zones'), {
    'min': 0,
    'max': 1,
    'palette': ['transparent', 'orange']
}, 'Medium Confidence (Top 20%)', False)

# Low Confidence (Top 30%)
Map.addLayer(alteration_potential_2023_2024.select('Low_Confidence_Zones'), {
    'min': 0,
    'max': 1,
    'palette': ['transparent', 'yellow']
}, 'Low Confidence (Top 30%)', False)

# All Zones Combined (Top 30% - for overview)
Map.addLayer(alteration_potential_2023_2024.select('All_Alteration_Zones'), {
    'min': 0,
    'max': 1,
    'palette': ['transparent', 'red']
}, 'All Alteration Zones (Top 30%)', False)

print("\n✓ Alteration-based potential maps created!")
print("\nConfidence Levels (Percentile-based):")
print("  🔴 Very High (Dark Red): Top 5% - Most significant alteration")
print("  🔴 High (Red): Top 10% - High confidence alteration")
print("  🟠 Medium (Orange): Top 20% - Moderate confidence")
print("  🟡 Low (Yellow): Top 30% - Potential alteration")
print("\n⚠️  IMPORTANT:")
print("  - These maps show ALTERATION-BASED potential only (spectral signatures)")
print("  - They do NOT incorporate structural controls (lineaments, faults)")
print("  - For FINAL MINERALIZATION MAPS with structural controls,")
print("    see mining-integrated.ipynb (standard practice in literature)")
print("=" * 60)
Map


Mineralisation maps created!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

## Temporal Change Detection


In [19]:
# Calculate change between the two time periods
def calculate_change(period1, period2):
    """Calculate change between two periods"""
    change = period2.subtract(period1).rename('Change')
    change_magnitude = change.abs().rename('Change_Magnitude')
    return change.addBands(change_magnitude)

# Calculate changes for each mineral type
clay_change = calculate_change(
    aster_2001_2006_clay.select('Clay_Index'),
    aster_2023_2024_clay.select('Clay_Index')
)

iron_change = calculate_change(
    aster_2001_2006_iron.select('Iron_Index'),
    aster_2023_2024_iron.select('Iron_Index')
)

carbonate_change = calculate_change(
    aster_2001_2006_carbonate.select('Carbonate_Index'),
    aster_2023_2024_carbonate.select('Carbonate_Index')
)

mineralisation_change = calculate_change(
    mineralisation_2001_2006.select('Mineralisation_Probability'),
    mineralisation_2023_2024.select('Mineralisation_Probability')
)

# Visualize changes
Map.addLayer(clay_change.select('Change'), {
    'min': -0.2,
    'max': 0.2,
    'palette': ['blue', 'white', 'red']
}, 'Clay Change (2023-2024 vs 2001-2006)')

Map.addLayer(iron_change.select('Change'), {
    'min': -0.3,
    'max': 0.3,
    'palette': ['blue', 'white', 'red']
}, 'Iron Change (2023-2024 vs 2001-2006)')

Map.addLayer(carbonate_change.select('Change'), {
    'min': -0.1,
    'max': 0.1,
    'palette': ['blue', 'white', 'red']
}, 'Carbonate Change (2023-2024 vs 2001-2006)')

Map.addLayer(mineralisation_change.select('Change'), {
    'min': -0.5,
    'max': 0.5,
    'palette': ['blue', 'white', 'red']
}, 'Mineralisation Change (2023-2024 vs 2001-2006)')

print("Temporal change analysis completed!")
Map


Temporal change analysis completed!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

## Export Results


In [ ]:
# Export alteration and mineralisation maps
# Note: Uncomment and run to export to Google Drive

export_params = {
    'image': None,  # Will be set for each export
    'description': None,  # Will be set for each export
    'scale': 30,  # 30m resolution
    'region': study_area_buffered,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13
}

# Export functions (uncomment to use)
def export_alteration_maps():
    """Export all alteration maps"""
    # Period 1
    geemap.ee_export_image(
        aster_2001_2006_clay.select(['Clay_Index', 'AlOH_Index', 'Kaolinite_Index']),
        filename='aster_alteration_clay_2001_2006.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        aster_2001_2006_iron.select(['Iron_Index', 'Ferrous_Index', 'Ferric_Index']),
        filename='aster_alteration_iron_2001_2006.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        aster_2001_2006_carbonate.select(['Carbonate_Index', 'Calcite_Index', 'Dolomite_Index']),
        filename='aster_alteration_carbonate_2001_2006.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    # Period 2
    geemap.ee_export_image(
        aster_2023_2024_clay.select(['Clay_Index', 'AlOH_Index', 'Kaolinite_Index']),
        filename='aster_alteration_clay_2023_2024.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        aster_2023_2024_iron.select(['Iron_Index', 'Ferrous_Index', 'Ferric_Index']),
        filename='aster_alteration_iron_2023_2024.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        aster_2023_2024_carbonate.select(['Carbonate_Index', 'Calcite_Index', 'Dolomite_Index']),
        filename='aster_alteration_carbonate_2023_2024.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )

def export_mineralisation_maps():
    """Export mineralisation maps"""
    geemap.ee_export_image(
        mineralisation_2001_2006,
        filename='aster_mineralisation_2001_2006.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        mineralisation_2023_2024,
        filename='aster_mineralisation_2023_2024.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )

def export_change_maps():
    """Export change detection maps"""
    geemap.ee_export_image(
        clay_change,
        filename='aster_clay_change.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        iron_change,
        filename='aster_iron_change.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        carbonate_change,
        filename='aster_carbonate_change.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        mineralisation_change,
        filename='aster_mineralisation_change.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )

print("Export functions defined. Uncomment and call to export maps.")
print("Example: export_alteration_maps()")
